# <img width=400 src="http://www.numpy.org/_static/numpy_logo.png" alt="Numpy"/>


## Why do we need numpy?

* You may have heard "Python is slow", this is true when it concerns looping over many small python objects
* Python is dynamically typed and everything is an object, even an `int`. There are no primitive types.
* Numpy's main feature is the `ndarray` class, a fixed length, homogeniously typed array class.
* Numpy implements a lot of functionality in fast c, cython and fortran code to work on these arrays
* python with vectorized operations using numpy can be blazingly fast

See: [Python is not C](https://www.ibm.com/developerworks/community/blogs/jfp/entry/Python_Is_Not_C?lang=en)

But the most important reason:

* More beautiful code

In [1]:
import numpy as np

## More beautiful code through vectorisation

pure python with list comprehension

In [2]:
voltages = [10.1, 15.1, 9.5]
currents = [1.2, 2.4, 5.2]

resistances = [U * I for U, I in zip(voltages, currents)]
resistances

[12.12, 36.239999999999995, 49.4]

Using numpy

In [3]:
U = np.array([10.1, 15.1, 9.5])
I = np.array([1.2, 2.4, 5.2])

R = U * I
R

array([ 12.12,  36.24,  49.4 ])

### Finding the point with the smallest distance

In [4]:
import math

def euclidean_distance(p1, p2):
    return math.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

point = (1, 2)
points = [(3, 2), (4, 2), (3, 0)]

min_distance = float('inf')
for other in points:
    distance = euclidean_distance(point, other)
    if distance < min_distance:
        closest = other
        min_distance = distance 

print(min_distance, closest)

2.0 (3, 2)


In [5]:
point = np.array([1, 2])
points = np.array([(3, 2), (4, 2), (3, 0)])

distance = np.linalg.norm(point - points, axis=1)
idx = np.argmin(distance)

print(distance[idx], points[idx])

2.0 [3 2]


## Small example timings

In [6]:
import math


def var(data):
    '''
    knuth's algorithm for one-pass calculation of the variance
    Avoids rounding errors of large numbers when doing the naive
    approach of `sum(v**2 for v in data) - sum(v)**2`
    '''
    
    n = 0
    mean = 0.0
    m2 = 0.0
    
    if len(data) < 2:
        return float('nan')

    for value in data:
        n += 1
        delta = value - mean
        mean += delta / n
        delta2 = value - mean
        m2 += delta * delta2

    return m2 / n 

In [7]:
%%timeit

l = list(range(1000))
var(l)

246 µs ± 5.06 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [8]:
%%timeit

a = np.arange(1000)  # array with numbers 0,...,999

np.var(a)

30.5 µs ± 241 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


## Basic math: vectorized

Operations on numpy arrays work vectorized, element-by-element

** Lose your loops **

In [9]:
# create a numpy array from a python a python list
a = np.array([1.0, 3.5, 7.1, 4, 6])

In [10]:
2 * a

array([  2. ,   7. ,  14.2,   8. ,  12. ])

In [11]:
a**2

array([  1.  ,  12.25,  50.41,  16.  ,  36.  ])

In [12]:
a**a

array([  1.00000000e+00,   8.02117802e+01,   1.10645633e+06,
         2.56000000e+02,   4.66560000e+04])

In [13]:
np.cos(a)

array([ 0.54030231, -0.93645669,  0.68454667, -0.65364362,  0.96017029])

**Attention: You need the `cos` from numpy!**

In [14]:
math.cos(a)

TypeError: only length-1 arrays can be converted to Python scalars

Most normal python functions with basic operators like `*`, `+`, `**` simply work because
of operator overloading:

In [15]:
def poly(x):
    return x + 2 * x**2 - x**3

poly(a)

array([   2.   ,  -14.875, -249.991,  -28.   , -138.   ])

In [16]:
poly(np.pi)

-8.125475224531307

## Useful properties

In [17]:
len(a)

5

In [18]:
a.shape

(5,)

In [19]:
a.dtype

dtype('float64')

In [20]:
a.ndim

1

## Arbitrary dimension arrays

In [21]:
# two-dimensional array
y = np.array([[1, 2, 3],
              [4, 5, 6],
              [7, 8, 9]])

y + y

array([[ 2,  4,  6],
       [ 8, 10, 12],
       [14, 16, 18]])

In [22]:
## since python 3.5 @ is matrix product
y @ y

array([[ 30,  36,  42],
       [ 66,  81,  96],
       [102, 126, 150]])

In [23]:
# Broadcasting, changing array dimensions to fit the larger one

y + np.array([1, 2, 3])

array([[ 2,  4,  6],
       [ 5,  7,  9],
       [ 8, 10, 12]])

## Reduction operations

Numpy has many operations, which reduce dimensionality of arrays

In [24]:
x = np.random.normal(0, 1, 10)

In [25]:
np.sum(x)

-0.024680822025888283

In [26]:
np.prod(x)

0.0081579549254615063

In [27]:
np.mean(x)

-0.0024680822025888284

Standard Deviation

In [28]:
np.std(x)

1.3637117096754443

Standard error of the mean

In [29]:
np.std(x, ddof=1) / np.sqrt(len(x))

0.45457056989181471

Sample Standard Deviation

In [30]:
np.std(x, ddof=1)

1.4374783581388946

Difference between neighbor elements

In [31]:
z = np.arange(10)**2
np.diff(z)

array([ 1,  3,  5,  7,  9, 11, 13, 15, 17])

### Reductions on multi-dimensional arrays


In [32]:
array2d = np.arange(20).reshape(4, 5)

array2d

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14],
       [15, 16, 17, 18, 19]])

In [33]:
np.sum(array2d, axis=0)

array([30, 34, 38, 42, 46])

In [34]:
np.mean(array2d, axis=1)

array([  2.,   7.,  12.,  17.])

## Exercise 1

Write a function that calculates the analytical linear regression for a set of
x and y values.

Reminder:

$$ f(x) = a \cdot x + b$$

with 

$$
\hat{b} = \frac{\mathrm{Cov}(x, y)}{\mathrm{Var}(x)} \\
\hat{a} = \bar{y} - \hat{b} \cdot \hat{x}
$$

In [ ]:
# %load 04_01_numpy_solutions/exercise_linear.py

In [ ]:
x = np.linspace(0, 1, 50)
y = 5 * np.random.normal(x, 0.1) + 2  # see section on random numbers later

print(linear_regression(x, y))

## Helpers for creating arrays

In [35]:
np.zeros(10)

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [36]:
np.ones((5, 2))

array([[ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.]])

In [37]:
np.full(5, np.nan)

array([ nan,  nan,  nan,  nan,  nan])

In [38]:
np.empty(5)  # attention, uninitialised memory, be carefull

array([ 0.,  0.,  0.,  0.,  0.])

In [39]:
np.linspace(0, 1, 11)

array([ 0. ,  0.1,  0.2,  0.3,  0.4,  0.5,  0.6,  0.7,  0.8,  0.9,  1. ])

In [40]:
# like range() for arrays:
np.arange(0, 10)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [41]:
np.logspace(-4, 5, 10)

array([  1.00000000e-04,   1.00000000e-03,   1.00000000e-02,
         1.00000000e-01,   1.00000000e+00,   1.00000000e+01,
         1.00000000e+02,   1.00000000e+03,   1.00000000e+04,
         1.00000000e+05])

## Numpy Indexing

* Element access
* Slicing

In [42]:
x = np.arange(0, 10)

# like lists:
x[4]

4

In [43]:
# all elements with indices ≥1 and <4:
x[1:4]

array([1, 2, 3])

In [44]:
# negative indices count from the end
x[-1], x[-2]

(9, 8)

In [45]:
# combination:
x[3:-2]

array([3, 4, 5, 6, 7])

In [46]:
# step size
x[::2]

array([0, 2, 4, 6, 8])

In [47]:
# trick for reversal: negative step
x[::-1]

array([9, 8, 7, 6, 5, 4, 3, 2, 1, 0])

In [48]:
y = np.array([x, x + 10, x + 20, x + 30])
y

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
       [30, 31, 32, 33, 34, 35, 36, 37, 38, 39]])

In [49]:
# comma between indices
y[3, 2:-1]

array([32, 33, 34, 35, 36, 37, 38])

In [50]:
# only one index ⇒ one-dimensional array
y[2]

array([20, 21, 22, 23, 24, 25, 26, 27, 28, 29])

In [51]:
# other axis: (: alone means the whole axis)
y[:, 3]

array([ 3, 13, 23, 33])

In [52]:
# inspecting the number of elements per axis:
y.shape

(4, 10)

# Changing array content

In [53]:
y

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
       [30, 31, 32, 33, 34, 35, 36, 37, 38, 39]])

In [54]:
y[:, 3] = 0
y

array([[ 0,  1,  2,  0,  4,  5,  6,  7,  8,  9],
       [10, 11, 12,  0, 14, 15, 16, 17, 18, 19],
       [20, 21, 22,  0, 24, 25, 26, 27, 28, 29],
       [30, 31, 32,  0, 34, 35, 36, 37, 38, 39]])

Using slices on both sides

In [55]:
y[:,0] = x[3:7]
y

array([[ 3,  1,  2,  0,  4,  5,  6,  7,  8,  9],
       [ 4, 11, 12,  0, 14, 15, 16, 17, 18, 19],
       [ 5, 21, 22,  0, 24, 25, 26, 27, 28, 29],
       [ 6, 31, 32,  0, 34, 35, 36, 37, 38, 39]])

Transposing inverts the order of the dimensions

In [56]:
y

array([[ 3,  1,  2,  0,  4,  5,  6,  7,  8,  9],
       [ 4, 11, 12,  0, 14, 15, 16, 17, 18, 19],
       [ 5, 21, 22,  0, 24, 25, 26, 27, 28, 29],
       [ 6, 31, 32,  0, 34, 35, 36, 37, 38, 39]])

In [57]:
y.shape

(4, 10)

In [58]:
y.T

array([[ 3,  4,  5,  6],
       [ 1, 11, 21, 31],
       [ 2, 12, 22, 32],
       [ 0,  0,  0,  0],
       [ 4, 14, 24, 34],
       [ 5, 15, 25, 35],
       [ 6, 16, 26, 36],
       [ 7, 17, 27, 37],
       [ 8, 18, 28, 38],
       [ 9, 19, 29, 39]])

In [59]:
y.T.shape

(10, 4)

# Masks

* A boolean array can be used to select only the element where it contains `True`.
* Very powerfull tool to select certain elements that fullfill a certain condition

In [60]:
a = np.linspace(0, 2, 11)
b = np.random.normal(0, 1, 11)

print(b >= 0)
print(a[b >= 0])

[ True  True False False False False False False  True  True False]
[ 0.   0.2  1.6  1.8]


In [61]:
a[b < 0] = 0
a

array([ 0. ,  0.2,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  1.6,  1.8,  0. ])

### Random numbers

* numpy has a larger number of distributions builtin

In [62]:
np.random.uniform(0, 1, 5)

array([ 0.25654407,  0.32331758,  0.13005964,  0.77651708,  0.43568895])

In [63]:
np.random.normal(5, 10, 5)

array([ 18.51156958,  20.70294055,  28.79179598,  20.25547234,  -0.80456926])

## Calculating pi through monte-carlo simulation

* We draw random numbers in a square with length of the sides of 2
* We count the points which are inside the circle of radius 1

The area of the square is

$$
A_\mathrm{square} = a^2 = 4
$$

The area of the circle is
$$
A_\mathrm{circle} = \pi r^2 = \pi
$$

With 
$$
\frac{n_\mathrm{circle}}{n_\mathrm{square}} = \frac{A_\mathrm{circle}}{A_\mathrm{square}}
$$
We can calculate pi:

$$
\pi = 4 \frac{n_\mathrm{circle}}{n_\mathrm{square}}
$$

In [64]:
n_square = 1000

x = np.random.uniform(-1, 1, n_square)
y = np.random.uniform(-1, 1, n_square)

radius = np.sqrt(x**2 + y**2)

n_circle = np.sum(radius <= 1.0)

print(4 * n_circle / n_square)

3.232


## Exercise

1. Draw 10000 gaussian random numbers with mean of $\mu = 2$ and standard deviation of $\sigma = 3$
2. Calculate the mean and the standard deviation of the sample
3. What percentage of the numbers are outside of $[\mu - \sigma, \mu + \sigma]$?
4. How many of the numbers are $> 0$?
5. Calculate the mean and the standard deviation of all numbers ${} > 0$

In [ ]:
# %load 04_01_numpy_solutions/exercise_gaussian.py

## Exercise

Monte-Carlo uncertainty propagation

* The hubble constant as measured by PLANCK is
$$
H_0 = (67.74 \pm 0.47)\,\frac{\mathrm{km}}{\mathrm{s}\cdot\mathrm{Mpc}}
$$

* Estimate mean and the uncertainty of the velocity of a galaxy which is measured to be $(500 \pm 100)\,\mathrm{Mpc}$ away
using monte carlo methods

In [ ]:
# %load 04_01_numpy_solutions/exercise_hubble.py

## Simple io functions

In [65]:
idx = np.arange(100)
x = np.random.normal(0, 1, 100)
y = np.random.normal(0, 1, 100)
n = np.random.poisson(20, 100)

In [66]:
idx.shape, x.shape, y.shape, n.shape

((100,), (100,), (100,), (100,))

In [67]:
np.savetxt(
    'data.csv',
    np.column_stack([idx, x, y, n]),
)

In [68]:
!head data.csv

0.000000000000000000e+00 -1.529859849792397641e+00 -1.346633221801033509e+00 2.200000000000000000e+01
1.000000000000000000e+00 4.775451075390010902e-01 -1.306694848450834945e+00 2.600000000000000000e+01
2.000000000000000000e+00 -1.690626910510918679e-01 6.779331676877842217e-01 2.400000000000000000e+01
3.000000000000000000e+00 1.121437405207876825e+00 7.708619707025609058e-01 1.600000000000000000e+01
4.000000000000000000e+00 -2.030737918333696754e-02 5.706135626028785435e-02 1.900000000000000000e+01
5.000000000000000000e+00 -5.908230846100721162e-02 -3.212971407863245421e-01 2.500000000000000000e+01
6.000000000000000000e+00 1.127029026127312372e+00 -1.179714443119899592e+00 2.100000000000000000e+01
7.000000000000000000e+00 -1.474054551318914330e-01 7.510778439090134428e-01 1.200000000000000000e+01
8.000000000000000000e+00 3.676016464617179458e-01 -2.099218886099579462e-01 1.800000000000000000e+01
9.000000000000000000e+00 -1.618980010227883071e+00 -5.598392315439133515e-01 2.70000000000

In [69]:
# Load back the data, unpack=True is needed to read the data columnwise and not row-wise
idx, x, y, n = np.genfromtxt('data.csv', unpack=True)

idx.dtype, x.dtype

(dtype('float64'), dtype('float64'))

### Problems

* Everything is a float
* Way larger file than necessary because of too much digits for floats
* No column names

## Numpy recarrays

* Numpy recarrays can store columns of different types
* Rows are addressed by integer index
* Columns are addressed by strings

Solution for our io problem → Column names, different types

In [70]:
data = np.savetxt(
    'data.csv',
    np.column_stack([idx, x, y, n]),
    delimiter=',', # true csv file
    header=','.join(['idx', 'x', 'y', 'n']),
    fmt=['%d', '%.4g', '%.4g', '%d'],
)

In [71]:
!head data.csv

# idx,x,y,n
0,-1.53,-1.347,22
1,0.4775,-1.307,26
2,-0.1691,0.6779,24
3,1.121,0.7709,16
4,-0.02031,0.05706,19
5,-0.05908,-0.3213,25
6,1.127,-1.18,21
7,-0.1474,0.7511,12
8,0.3676,-0.2099,18


In [72]:
data = np.genfromtxt(
    'data.csv',
    names=True, # load column names from first row
    dtype=None, # Automagically determince best data type for each column
    delimiter=',',
)

In [73]:
data[0]

(0, -1.53, -1.347, 22)

In [74]:
data['x']

array([-1.53    ,  0.4775  , -0.1691  ,  1.121   , -0.02031 , -0.05908 ,
        1.127   , -0.1474  ,  0.3676  , -1.619   ,  1.949   ,  1.153   ,
       -0.9683  , -1.132   ,  1.932   , -0.09299 ,  0.2614  , -1.056   ,
       -1.657   ,  0.2081  , -1.002   , -0.7782  ,  0.008113,  1.471   ,
       -0.7393  , -0.4474  , -0.9282  , -0.3941  , -0.1501  ,  0.5381  ,
        0.5312  ,  0.1197  ,  1.545   ,  1.206   ,  0.5824  ,  0.129   ,
       -0.4185  , -1.229   , -1.687   ,  1.022   ,  0.6689  ,  0.2608  ,
       -0.8049  , -1.266   ,  0.2501  ,  0.1878  , -1.412   ,  0.8199  ,
        0.06454 ,  0.2893  ,  1.325   ,  0.9386  , -2.475   , -0.6142  ,
        0.7134  , -0.08336 , -0.848   ,  1.103   ,  1.617   , -1.083   ,
        0.1603  , -0.2387  , -0.9139  ,  0.8258  , -0.8514  , -1.275   ,
        1.052   , -1.83    , -1.078   ,  0.578   ,  0.1427  ,  0.297   ,
       -0.04243 , -0.3932  , -0.1027  ,  0.4139  , -0.9673  ,  0.2329  ,
       -0.4204  ,  1.651   , -1.811   ,  0.5557  , 

In [75]:
data.dtype

dtype([('idx', '<i8'), ('x', '<f8'), ('y', '<f8'), ('n', '<i8')])

## Linear algebra

Numpy offers a lot of linear algebra functionality, mostly wrapping LAPACK

In [76]:
# symmetrix matrix, use eigh
mat = np.array([
    [4, 2, 0],
    [2, 1, -3],
    [0, -3, 4]
])

eig_vals, eig_vecs = np.linalg.eig(mat)

eig_vals, eig_vecs

(array([-1.40512484,  4.        ,  6.40512484]),
 array([[  3.07818468e-01,   8.32050294e-01,  -4.61454330e-01],
        [ -8.31898624e-01,  -1.93604245e-16,  -5.54927635e-01],
        [ -4.61727702e-01,   5.54700196e-01,   6.92181495e-01]]))

In [77]:
np.linalg.inv(mat)

array([[ 0.13888889,  0.22222222,  0.16666667],
       [ 0.22222222, -0.44444444, -0.33333333],
       [ 0.16666667, -0.33333333,  0.        ]])

## Numpy matrices

Numpy also has a matrix class, with operator overloading suited for matrices

In [78]:
mat = np.matrix(mat)

In [79]:
mat.T

matrix([[ 4,  2,  0],
        [ 2,  1, -3],
        [ 0, -3,  4]])

In [80]:
mat * mat

matrix([[ 20,  10,  -6],
        [ 10,  14, -15],
        [ -6, -15,  25]])

In [81]:
mat * 5

matrix([[ 20,  10,   0],
        [ 10,   5, -15],
        [  0, -15,  20]])

In [82]:
mat.I

matrix([[ 0.13888889,  0.22222222,  0.16666667],
        [ 0.22222222, -0.44444444, -0.33333333],
        [ 0.16666667, -0.33333333,  0.        ]])

In [83]:
mat * np.matrix([1, 2, 3]).T

matrix([[ 8],
        [-5],
        [ 6]])